In [ ]:
import pandas as pd
import json
from tqdm import tqdm
import numpy as np

# Load the recipes dataset into a DataFrame
df = pd.read_csv('recipes_data.csv')

In [ ]:
df

,title,ingredients,directions,link,source,NER,site
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""bite size shredded rice biscuits"", ""vanilla""...",www.cookbooks.com
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""cream of mushroom soup"", ""beef"", ""sour cream...",www.cookbooks.com
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""pepper"", ""cream cheese"", ""gar...",www.cookbooks.com
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken gravy"", ""cream of mushroom soup"", ""c...",www.cookbooks.com
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""graham cracker crumbs"", ""powdered sugar"", ""p...",www.cookbooks.com
...,...,...,...,...,...,...,...
2231137,Sunny's Fake Crepes,"[""1/2 cup chocolate hazelnut spread (recommend...","[""Spread hazelnut spread on 1 side of each tor...",www.foodnetwork.com/recipes/sunny-anderson/sun...,Recipes1M,"[""chocolate hazelnut spread"", ""marshmallows"", ...",www.foodnetwork.com
2231138,Devil Eggs,"[""1 dozen eggs"", ""1 paprika"", ""1 salt and pepp...","[""Boil eggs on medium for 30mins."", ""Then cool...",cookpad.com/us/recipes/355411-devil-eggs,Recipes1M,"[""choice"", ""miracle whip"", ""eggs"", ""relish"", ""...",cookpad.com
2231139,Extremely Easy and Quick - Namul Daikon Salad,"[""150 grams Daikon radish"", ""1 tbsp Sesame oil...","[""Julienne the daikon and squeeze out the exce...",cookpad.com/us/recipes/153324-extremely-easy-a...,Recipes1M,"[""soy sauce"", ""radish"", ""white sesame seeds"", ...",cookpad.com
2231140,Pan-Roasted Pork Chops With Apple Fritters,"[""1 cup apple cider"", ""6 tablespoons sugar"", ""...","[""In a large bowl, mix the apple cider with 4 ...",cooking.nytimes.com/recipes/1015164,Recipes1M,"[""apple cider"", ""egg"", ""sugar"", ""freshly groun...",cooking.nytimes.com


In [ ]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2231142 entries, 0 to 2231141
Data columns (total 7 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   title        object
 1   ingredients  object
 2   directions   object
 3   link         object
 4   source       object
 5   NER          object
 6   site         object
dtypes: object(7)
memory usage: 2.9 GB


In [ ]:
# exp_df = df.sample(n=500000, random_state=74, ignore_index=True)
#Take out the brackets and " from ingredients
df['NER'] = df['NER'].str.replace('[','').str.replace(']','').str.replace('"','')
# exp_df['ingredients'] = exp_df['ingredients'].str.replace('[','').str.replace(']','').str.replace('"','')  
df['ingredients'] = df['ingredients'].str.replace('[\[\]"]', '', regex=True)

def create_prompt(row):
    ingredients = row['NER'].split(',')
    prompt = f"The Ingredients of the recipe: {', '.join(ingredients)}"
    return prompt

for index, row in df.iterrows():
    prompt = create_prompt(row)
    df.at[index, 'prompt'] = prompt

In [ ]:
df

,title,ingredients,directions,link,source,NER,site,prompt
0,No-Bake Nut Cookies,"1 c. firmly packed brown sugar, 1/2 c. evapora...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"bite size shredded rice biscuits, vanilla, bro...",www.cookbooks.com,The Ingredients of the recipe: bite size shred...
1,Jewell Ball'S Chicken,"1 small jar chipped beef, cut up, 4 boned chic...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"cream of mushroom soup, beef, sour cream, chic...",www.cookbooks.com,The Ingredients of the recipe: cream of mushro...
2,Creamy Corn,"2 (16 oz.) pkg. frozen corn, 1 (8 oz.) pkg. cr...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"frozen corn, pepper, cream cheese, garlic powd...",www.cookbooks.com,"The Ingredients of the recipe: frozen corn, p..."
3,Chicken Funny,"1 large whole chicken, 2 (10 1/2 oz.) cans chi...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"chicken gravy, cream of mushroom soup, chicken...",www.cookbooks.com,"The Ingredients of the recipe: chicken gravy, ..."
4,Reeses Cups(Candy),"1 c. peanut butter, 3/4 c. graham cracker crum...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"graham cracker crumbs, powdered sugar, peanut ...",www.cookbooks.com,The Ingredients of the recipe: graham cracker ...
...,...,...,...,...,...,...,...,...
2231137,Sunny's Fake Crepes,1/2 cup chocolate hazelnut spread (recommended...,"[""Spread hazelnut spread on 1 side of each tor...",www.foodnetwork.com/recipes/sunny-anderson/sun...,Recipes1M,"chocolate hazelnut spread, marshmallows, hazel...",www.foodnetwork.com,The Ingredients of the recipe: chocolate hazel...
2231138,Devil Eggs,"1 dozen eggs, 1 paprika, 1 salt and pepper to ...","[""Boil eggs on medium for 30mins."", ""Then cool...",cookpad.com/us/recipes/355411-devil-eggs,Recipes1M,"choice, miracle whip, eggs, relish, paprika, salt",cookpad.com,"The Ingredients of the recipe: choice, miracl..."
2231139,Extremely Easy and Quick - Namul Daikon Salad,"150 grams Daikon radish, 1 tbsp Sesame oil, 1 ...","[""Julienne the daikon and squeeze out the exce...",cookpad.com/us/recipes/153324-extremely-easy-a...,Recipes1M,"soy sauce, radish, white sesame seeds, sesame ...",cookpad.com,"The Ingredients of the recipe: soy sauce, rad..."
2231140,Pan-Roasted Pork Chops With Apple Fritters,"1 cup apple cider, 6 tablespoons sugar, 4 tabl...","[""In a large bowl, mix the apple cider with 4 ...",cooking.nytimes.com/recipes/1015164,Recipes1M,"apple cider, egg, sugar, freshly ground black ...",cooking.nytimes.com,"The Ingredients of the recipe: apple cider, e..."


In [ ]:
def count_ingredients(s):
    try:
        s_split = s.split(', ')
        s_split_len = len(s_split)
    except:
        s_split_len = 1
    return s_split_len

def count_directions(s):
    return len(json.loads(s))

df['directions_len'] = df['directions'].str.len()
df['directions_count'] = df['directions'].apply(count_directions)
df['NER_count'] = df['NER'].apply(count_ingredients)

In [ ]:
df

,title,ingredients,directions,link,source,NER,site,prompt,directions_len,directions_count,NER_count
0,No-Bake Nut Cookies,"1 c. firmly packed brown sugar, 1/2 c. evapora...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"bite size shredded rice biscuits, vanilla, bro...",www.cookbooks.com,The Ingredients of the recipe: bite size shred...,357,6,6
1,Jewell Ball'S Chicken,"1 small jar chipped beef, cut up, 4 boned chic...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"cream of mushroom soup, beef, sour cream, chic...",www.cookbooks.com,The Ingredients of the recipe: cream of mushro...,175,3,4
2,Creamy Corn,"2 (16 oz.) pkg. frozen corn, 1 (8 oz.) pkg. cr...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"frozen corn, pepper, cream cheese, garlic powd...",www.cookbooks.com,"The Ingredients of the recipe: frozen corn, p...",171,1,6
3,Chicken Funny,"1 large whole chicken, 2 (10 1/2 oz.) cans chi...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"chicken gravy, cream of mushroom soup, chicken...",www.cookbooks.com,"The Ingredients of the recipe: chicken gravy, ...",394,6,4
4,Reeses Cups(Candy),"1 c. peanut butter, 3/4 c. graham cracker crum...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"graham cracker crumbs, powdered sugar, peanut ...",www.cookbooks.com,The Ingredients of the recipe: graham cracker ...,229,3,5
...,...,...,...,...,...,...,...,...,...,...,...
2231137,Sunny's Fake Crepes,1/2 cup chocolate hazelnut spread (recommended...,"[""Spread hazelnut spread on 1 side of each tor...",www.foodnetwork.com/recipes/sunny-anderson/sun...,Recipes1M,"chocolate hazelnut spread, marshmallows, hazel...",www.foodnetwork.com,The Ingredients of the recipe: chocolate hazel...,880,12,5
2231138,Devil Eggs,"1 dozen eggs, 1 paprika, 1 salt and pepper to ...","[""Boil eggs on medium for 30mins."", ""Then cool...",cookpad.com/us/recipes/355411-devil-eggs,Recipes1M,"choice, miracle whip, eggs, relish, paprika, salt",cookpad.com,"The Ingredients of the recipe: choice, miracl...",423,7,6
2231139,Extremely Easy and Quick - Namul Daikon Salad,"150 grams Daikon radish, 1 tbsp Sesame oil, 1 ...","[""Julienne the daikon and squeeze out the exce...",cookpad.com/us/recipes/153324-extremely-easy-a...,Recipes1M,"soy sauce, radish, white sesame seeds, sesame ...",cookpad.com,"The Ingredients of the recipe: soy sauce, rad...",115,3,5
2231140,Pan-Roasted Pork Chops With Apple Fritters,"1 cup apple cider, 6 tablespoons sugar, 4 tabl...","[""In a large bowl, mix the apple cider with 4 ...",cooking.nytimes.com/recipes/1015164,Recipes1M,"apple cider, egg, sugar, freshly ground black ...",cooking.nytimes.com,"The Ingredients of the recipe: apple cider, e...",2315,22,26


In [ ]:
filter_nb_directions = 8
filter_nb_ingredients = 1
filterr = (df['directions_count'] >= filter_nb_directions) & (df['NER_count'] >= filter_nb_ingredients)
percentage_kept = (filterr.sum() / df.shape[0]) * 100
print('You keep %.2f%% of the data' % percentage_kept)

You keep 30.37% of the data


In [ ]:
df_filtered = df[filterr]

In [ ]:
df_filtered

,title,ingredients,directions,link,source,NER,site,prompt,directions_len,directions_count,NER_count
5,Cheeseburger Potato Soup,"6 baking potatoes, 1 lb. of extra lean ground ...","[""Wash potatoes; prick several times with a fo...",www.cookbooks.com/Recipe-Details.aspx?id=20115,Gathered,"sour cream, bacon, pepper, extra lean ground b...",www.cookbooks.com,"The Ingredients of the recipe: sour cream, ba...",904,16,10
9,Millionaire Pie,"1 large container Cool Whip, 1 large can crush...","[""Empty Cool Whip into a bowl."", ""Drain juice ...",www.cookbooks.com/Recipe-Details.aspx?id=794547,Gathered,"condensed milk, lemons, graham cracker crusts,...",www.cookbooks.com,"The Ingredients of the recipe: condensed milk,...",401,11,5
10,Double Cherry Delight,"1 (17 oz.) can dark sweet pitted cherries, 1/2...","[""Drain cherries, measuring syrup."", ""Cut cher...",www.cookbooks.com/Recipe-Details.aspx?id=703381,Gathered,"flavor gelatin, dark sweet pitted cherries, ma...",www.cookbooks.com,"The Ingredients of the recipe: flavor gelatin,...",385,9,6
15,Fresh Strawberry Pie,"1 baked pie shell, 1 qt. cleaned strawberries,...","[""Mix water, cornstarch, sugar and salt in sau...",www.cookbooks.com/Recipe-Details.aspx?id=161321,Gathered,"sugar, shell, water, cleaned strawberries, cor...",www.cookbooks.com,"The Ingredients of the recipe: sugar, shell, ...",340,8,7
23,Spaghetti Sauce To Can,"1/2 bushel tomatoes, 1 c. oil, 1/4 c. minced g...","[""Cook ground or chopped peppers and onions in...",www.cookbooks.com/Recipe-Details.aspx?id=1059279,Gathered,"oil, oregano, sugar, tomatoes, tomato paste, i...",www.cookbooks.com,"The Ingredients of the recipe: oil, oregano, ...",372,8,10
...,...,...,...,...,...,...,...,...,...,...,...
2231133,Basil Chicken Parmigiana,"8 ounces uncooked rotini pasta, 24 ounces chic...","[""Prepare pasta according to package direction...",www.food.com/recipe/basil-chicken-parmigiana-2...,Recipes1M,"egg substitute, chicken breasts, pasta sauce, ...",www.food.com,"The Ingredients of the recipe: egg substitute,...",843,13,10
2231135,Mozzarella Meatball Sandwiches,1 loaf pepperidge farm frozen mozzarella garli...,"[""Heat the oven to 400F."", ""Remove the bread f...",www.food.com/recipe/mozzarella-meatball-sandwi...,Recipes1M,"italian sauce, bread, frozen meatballs",www.food.com,"The Ingredients of the recipe: italian sauce, ...",579,11,3
2231136,Curried Asparagus Vichyssoise,"1 sweet onion, chopped (Vidalia, Maui, etc.), ...","[""Cook the onion in butter in a medium saucepa...",www.food.com/recipe/curried-asparagus-vichysso...,Recipes1M,"then, sweet onion, curry powder, soy sauce, gr...",www.food.com,"The Ingredients of the recipe: then, sweet on...",972,13,12
2231137,Sunny's Fake Crepes,1/2 cup chocolate hazelnut spread (recommended...,"[""Spread hazelnut spread on 1 side of each tor...",www.foodnetwork.com/recipes/sunny-anderson/sun...,Recipes1M,"chocolate hazelnut spread, marshmallows, hazel...",www.foodnetwork.com,The Ingredients of the recipe: chocolate hazel...,880,12,5


In [ ]:
title_duplicated_dict = {}
for title_duplicated in tqdm(df_filtered[df_filtered['title'].duplicated(keep=False)].loc[:, 'title'].values):
    same_title_instances = df_filtered[df_filtered.loc[:, 'title'] == title_duplicated].index
    same_title_instances_ner_count = df_filtered.loc[same_title_instances, 'NER_count'].values
    title_duplicated_dict[title_duplicated] = sorted([(a, b) for a, b in zip(same_title_instances, same_title_instances_ner_count)], key=lambda x: x[1])

100%|██████████| 209925/209925 [3:53:18<00:00, 15.00it/s]  


In [ ]:
len(title_duplicated_dict)

43669

In [ ]:
duplicated_selected_indexes = []
for k, v in title_duplicated_dict.items():
    duplicated_selected_indexes.append(v[0][0]) # Only keep recipes containing as few ingredients as possible
duplicated_selected_indexes = np.array(duplicated_selected_indexes)

In [ ]:
len(duplicated_selected_indexes)

43669

In [ ]:
df_filtered_without_duplicated = df_filtered[~df_filtered['title'].duplicated(keep=False)]
df_filtered_without_duplicated.shape[0]

467761

In [ ]:
df_filtered_clean = pd.concat([df_filtered_without_duplicated, df_filtered.loc[duplicated_selected_indexes]])
df_filtered_clean.shape[0]

511430

In [ ]:
df_filtered_clean

,title,ingredients,directions,link,source,NER,site,prompt,directions_len,directions_count,NER_count
5,Cheeseburger Potato Soup,"6 baking potatoes, 1 lb. of extra lean ground ...","[""Wash potatoes; prick several times with a fo...",www.cookbooks.com/Recipe-Details.aspx?id=20115,Gathered,"sour cream, bacon, pepper, extra lean ground b...",www.cookbooks.com,"The Ingredients of the recipe: sour cream, ba...",904,16,10
10,Double Cherry Delight,"1 (17 oz.) can dark sweet pitted cherries, 1/2...","[""Drain cherries, measuring syrup."", ""Cut cher...",www.cookbooks.com/Recipe-Details.aspx?id=703381,Gathered,"flavor gelatin, dark sweet pitted cherries, ma...",www.cookbooks.com,"The Ingredients of the recipe: flavor gelatin,...",385,9,6
23,Spaghetti Sauce To Can,"1/2 bushel tomatoes, 1 c. oil, 1/4 c. minced g...","[""Cook ground or chopped peppers and onions in...",www.cookbooks.com/Recipe-Details.aspx?id=1059279,Gathered,"oil, oregano, sugar, tomatoes, tomato paste, i...",www.cookbooks.com,"The Ingredients of the recipe: oil, oregano, ...",372,8,10
30,Punch Bowl Fruit Salad,"2 large cans sliced peaches, 2 large cans frui...","[""In a 6-quart punch bowl mix all of the cans ...",www.cookbooks.com/Recipe-Details.aspx?id=1059877,Gathered,"yellow apples, purple grapes, fresh strawberri...",www.cookbooks.com,"The Ingredients of the recipe: yellow apples, ...",275,8,9
38,Pear-Lime Salad,"1 (16 oz.) can pear halves, undrained, 1 (3 oz...","[""Drain pears, reserving juice."", ""Bring juice...",www.cookbooks.com/Recipe-Details.aspx?id=174717,Gathered,"lemon yogurt, lime gelatin, pear halves, cream...",www.cookbooks.com,"The Ingredients of the recipe: lemon yogurt, ...",387,10,4
...,...,...,...,...,...,...,...,...,...,...,...
2226425,Zesty Blueberry Muffins,"1 12 cups flour, 14 teaspoon salt, 34 cup milk...","[""Grease and line 12 muffin tins."", ""Mix Flour...",www.food.com/recipe/zesty-blueberry-muffins-18...,Recipes1M,"egg, baking powder, sugar, blueberries, lemon ...",www.food.com,"The Ingredients of the recipe: egg, baking po...",345,10,9
2230675,Dirty Risotto,"5 cups reduced-sodium chicken broth, 2 tablesp...","[""In a medium saucepan, bring the broth to a s...",www.foodnetwork.com/recipes/giada-de-laurentii...,Recipes1M,"freshly grated parmesan, button mushrooms, oni...",www.foodnetwork.com,The Ingredients of the recipe: freshly grated ...,1078,14,13
2227526,Warm Chickpea Salad,"1 red onion, cut into wedges, 2 courgettes, th...","[""Preheat the oven to fan 200C/ conventional 2...",www.food.com/recipe/warm-chickpea-salad-193867,Recipes1M,"tomatoes, chickpeas, mixed herbs, red onion, f...",www.food.com,"The Ingredients of the recipe: tomatoes, chic...",684,9,8
2226616,Layered Mediterranean Salad,"3 cups plain yogurt, 2 English cucumbers, seed...","[""Make the cucumber layer: Drain the yogurt in...",www.foodnetwork.com/recipes/layered-mediterran...,Recipes1M,"tomatoes, red-wine vinegar, freshly ground bla...",www.foodnetwork.com,"The Ingredients of the recipe: tomatoes, red-...",1126,15,19


In [ ]:
df_filtered_clean.drop(['directions_len', 'directions_count', 'NER_count'], axis=1, inplace=True)

In [ ]:
df_filtered_clean.to_csv('df_filtered_clean.csv')

In [ ]:
df_filtered_clean

,title,ingredients,directions,link,source,NER,site,prompt
5,Cheeseburger Potato Soup,"6 baking potatoes, 1 lb. of extra lean ground ...","[""Wash potatoes; prick several times with a fo...",www.cookbooks.com/Recipe-Details.aspx?id=20115,Gathered,"sour cream, bacon, pepper, extra lean ground b...",www.cookbooks.com,"The Ingredients of the recipe: sour cream, ba..."
10,Double Cherry Delight,"1 (17 oz.) can dark sweet pitted cherries, 1/2...","[""Drain cherries, measuring syrup."", ""Cut cher...",www.cookbooks.com/Recipe-Details.aspx?id=703381,Gathered,"flavor gelatin, dark sweet pitted cherries, ma...",www.cookbooks.com,"The Ingredients of the recipe: flavor gelatin,..."
23,Spaghetti Sauce To Can,"1/2 bushel tomatoes, 1 c. oil, 1/4 c. minced g...","[""Cook ground or chopped peppers and onions in...",www.cookbooks.com/Recipe-Details.aspx?id=1059279,Gathered,"oil, oregano, sugar, tomatoes, tomato paste, i...",www.cookbooks.com,"The Ingredients of the recipe: oil, oregano, ..."
30,Punch Bowl Fruit Salad,"2 large cans sliced peaches, 2 large cans frui...","[""In a 6-quart punch bowl mix all of the cans ...",www.cookbooks.com/Recipe-Details.aspx?id=1059877,Gathered,"yellow apples, purple grapes, fresh strawberri...",www.cookbooks.com,"The Ingredients of the recipe: yellow apples, ..."
38,Pear-Lime Salad,"1 (16 oz.) can pear halves, undrained, 1 (3 oz...","[""Drain pears, reserving juice."", ""Bring juice...",www.cookbooks.com/Recipe-Details.aspx?id=174717,Gathered,"lemon yogurt, lime gelatin, pear halves, cream...",www.cookbooks.com,"The Ingredients of the recipe: lemon yogurt, ..."
...,...,...,...,...,...,...,...,...
2226425,Zesty Blueberry Muffins,"1 12 cups flour, 14 teaspoon salt, 34 cup milk...","[""Grease and line 12 muffin tins."", ""Mix Flour...",www.food.com/recipe/zesty-blueberry-muffins-18...,Recipes1M,"egg, baking powder, sugar, blueberries, lemon ...",www.food.com,"The Ingredients of the recipe: egg, baking po..."
2230675,Dirty Risotto,"5 cups reduced-sodium chicken broth, 2 tablesp...","[""In a medium saucepan, bring the broth to a s...",www.foodnetwork.com/recipes/giada-de-laurentii...,Recipes1M,"freshly grated parmesan, button mushrooms, oni...",www.foodnetwork.com,The Ingredients of the recipe: freshly grated ...
2227526,Warm Chickpea Salad,"1 red onion, cut into wedges, 2 courgettes, th...","[""Preheat the oven to fan 200C/ conventional 2...",www.food.com/recipe/warm-chickpea-salad-193867,Recipes1M,"tomatoes, chickpeas, mixed herbs, red onion, f...",www.food.com,"The Ingredients of the recipe: tomatoes, chic..."
2226616,Layered Mediterranean Salad,"3 cups plain yogurt, 2 English cucumbers, seed...","[""Make the cucumber layer: Drain the yogurt in...",www.foodnetwork.com/recipes/layered-mediterran...,Recipes1M,"tomatoes, red-wine vinegar, freshly ground bla...",www.foodnetwork.com,"The Ingredients of the recipe: tomatoes, red-..."


In [ ]:
nan_NER = []
for a, b in enumerate(df_filtered_clean['NER'].str.split(', ').tolist()):
    if isinstance(b, float):
        nan_NER.append(a)
nan_NER = np.array(nan_NER)

In [ ]:
df_filtered_clean = df_filtered_clean[~df_filtered_clean.index.isin(nan_NER)]

In [ ]:
df_filtered_clean.shape

In [ ]:
# df_filtered_clean.to_csv('./df_filtered_clean.csv')

In [3]:
import pandas as pd

exp_df= pd.read_csv('df_filtered_clean.csv')

In [ ]:
exp_df = df_filtered_clean

In [2]:
from sentence_transformers import SentenceTransformer
transformer = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

/Users/ricardofernandezgasca/Library/Python/3.9/lib/python/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
from tqdm import tqdm
import pickle

# Create an iterator to save the embedding in the DataFrame for each line of the recipes
embeddings = transformer.encode(exp_df['prompt'].tolist(), show_progress_bar=True, convert_to_tensor=True)
# Store sentences & embeddings on disc
with open("embeddings.pkl", "wb") as fOut:
    pickle.dump({"sentences": exp_df['prompt'], "embeddings": embeddings}, fOut, protocol=pickle.HIGHEST_PROTOCOL)

In [1]:
import pickle
#Load embeddings from disc
with open("embeddings_cleanData.pkl", "rb") as fIn:
    stored_data = pickle.load(fIn)
    sentences = stored_data['sentences']
    embeddings = stored_data['embeddings']

In [8]:

print(type(embeddings))

<class 'torch.Tensor'>


In [4]:
from sentence_transformers import util
import torch

#Inference example
query = "The ingredientes of the recipie: potato, peppers, onions, cheese, eggs"
query_embedding = transformer.encode(query, convert_to_tensor=True)

# We use cosine-similarity and torch.topk to find the highest 5 scores
top_k = 10
cos_scores = util.cos_sim(query_embedding, embeddings)[0]
top_results = torch.topk(cos_scores, k=top_k)
for score, idx in zip(top_results[0], top_results[1]):
        idx = idx.item()  # Convert PyTorch tensor to a Python integer
        print(exp_df.loc[idx, 'title'], "(Score: {:.4f})".format(score))

"Egg-stra" Special Skillet Dish (Score: 0.8096)
Country Morning Skillet (Score: 0.8016)
Emily's Four-Cheese Country Morning Skillet (Score: 0.8012)
Brunchtime Country Frittata #5FIX (Score: 0.7939)
K's Ultimate Breakfast Scramble #5FIX (Score: 0.7928)
Country Breakfast (Score: 0.7916)
Cheesy Yellow Pepper Country Morning Skillet (Score: 0.7904)
Down on the Farm Breakfast (Score: 0.7890)
Special Egg Skillet Dish (Score: 0.7879)
Cheesy Bacon, Egg & Potato Skillet (Score: 0.7879)


In [15]:
# Store the best indexes in a list
best_indexes = []
for score, idx in zip(top_results[0], top_results[1]):
    idx = idx.item()  # Convert PyTorch tensor to a Python integer
    best_indexes.append(idx)

In [16]:
#Get the top 3 indexes and delete them from the list
top3_indexes = best_indexes[:3]
del best_indexes[:3]

print(top3_indexes)

[285975, 25824, 363426]


In [19]:
import random
#Choose 3 random indexes from the remaining indexes 
random_indexes = random.sample(best_indexes, 3)
#Delete the random indexes from the list
for idx in random_indexes:
    best_indexes.remove(idx)
recipe_titles = [exp_df.iloc[idx]['title'] for idx in random_indexes]
#
print(recipe_titles)

['Potato And Pepper Frittata ', 'Wednesday Supper #5FIX', "Ida Mae'S Potato Salad"]


In [1]:
import csv

# Read the CSV file and extract ingredient strings
ingredient_list = []
with open('top-1k-ingredients (2).csv', 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    for row in reader:
        ingredient_string, _ = row[0].split(';')
        ingredient_list.append(ingredient_string)

# Write the lookup section to a YAML file
with open('ingredients_lookup.yml', 'w', encoding='utf-8') as file:
    file.write("lookup: ingredients\n")
    file.write("examples: |\n")
    for ingredient in ingredient_list:
        file.write(f"  - {ingredient}\n")


In [2]:
ingredients = ['oil', 'habaneros', 'garlic', 'bread', 'olive oil', 'garlic', 'bread', "olive", 'unsalted butter','butter']
filtered_ingredients = []
for ingredient in ingredients:
    if ingredient not in filtered_ingredients:
        # Check if the ingredient is a sub-word of another ingredient
        is_sub_word = False
        for other_ingredient in ingredients:
            if ingredient != other_ingredient and ingredient in other_ingredient:
                is_sub_word = True
                break
        if not is_sub_word:
            filtered_ingredients.append(ingredient)
print(filtered_ingredients)    

['habaneros', 'garlic', 'bread', 'olive oil', 'unsalted butter']


In [16]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ricardofernandezgasca/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [13]:
import numpy as np
import random
# Create a random boolean column in the data set to say if its vegetarian or not
exp_df['is_vegetarian'] = np.random.choice([True, False], exp_df.shape[0])

In [14]:
from sentence_transformers import util
import torch
import nltk
from nltk.stem import WordNetLemmatizer
# Function to lemmatize an ingredient
lemmatizer = WordNetLemmatizer()
def lemmatize_ingredient(ingredient):
    return lemmatizer.lemmatize(ingredient.lower())


# Look for recipies in the embeddings and filter ingredients 
filtered_recipes = []
exclude_ingredients = ['onions', 'eggs']
include_ingredients = ['potato', 'peppers', 'tomato', 'cheese']
vegetarian = True #include a vegetarian flag
if vegetarian:
    query = "The ingredientes of the vegetarian recipie are:" + ', '.join(include_ingredients)
else:
    query = "The ingredientes of the recipie:" + ', '.join(include_ingredients)

query_embedding = transformer.encode(query, convert_to_tensor=True)

top_k = 200
cos_scores = util.cos_sim(query_embedding, embeddings)[0]
top_results = torch.topk(cos_scores, k=top_k)
filtered_recipes = []
best_indexes = []

for score, idx in zip(top_results[0], top_results[1]):
    idx = idx.item()  # Convert PyTorch tensor to a Python integer
    recipe = exp_df.loc[idx]
    recipe_ingredients = recipe['NER'].split(', ')
    # Check if the recipe contains any excluded ingredients
    contains_excluded = False
    for ingredient in exclude_ingredients:
        if any(lemmatize_ingredient(ingredient) in lemmatize_ingredient(recipe_ingredient) for recipe_ingredient in recipe_ingredients):
            print(f"Excluded ingredient found in recipe {recipe['title']}: {ingredient}", recipe_ingredients)
            contains_excluded = True
            break
    else:
        if vegetarian:
            # Check in the dataset if is_vegetarian is True
            if recipe['is_vegetarian']==True:
                filtered_recipes.append(recipe)
                best_indexes.append(idx)
            else:
                print(f"Recipe {recipe['title']} is not vegetarian")
                continue
        else:    
            filtered_recipes.append(recipe)
            best_indexes.append(idx)

print("Filtered recipes:", len(filtered_recipes))
print("Good indexes:", best_indexes)

Excluded ingredient found in recipe Vegetarian Garlic Hamburger Pizza: eggs ['vegetarian beef substitute', 'pizza seasoning', 'crust', 'roma tomato', 'veggie shreds cheese', 'garlic', 'olive oil']
Excluded ingredient found in recipe Vegetarian Pea Soup With Roasted Vegetables: onions ['vegetarian beef broth', 'peas', 'pepper', 'stalks celery', 'parsnips', 'onion', 'green pepper', 'carrots', 'olive oil', 'salt']
Excluded ingredient found in recipe Sweet & Sour Vegetarian Golabki (Stuffed Cabbage Rolls): onions ['vegetarian beef substitute', 'bread', 'egg', 'black pepper', 'head green cabbage', 'brown sugar', 'garlic', 'onion', 'lemon', 'mushroom', 'brown rice', 'ketchup', 'tomato sauce', 'filling', 'vegetarian beef', 'salt']
Excluded ingredient found in recipe New Potato, Fennel, Onion and Mushroom Pie with Mozzarella: onions ['potatoes', 'mozzarella cheese', 'mushrooms', 'onions', 'parsley', 'garlic', 'bread crumbs', 'olive oil', 'cheddar cheese vegetarian']
Excluded ingredient found i

In [8]:
print("Top recipes:")
for idx in best_indexes:
    print(exp_df.loc[idx, 'title'])
    

Top recipes:
Baked Potatoes With Baked Beans Recipe
Cheesy Potato Beans Delight
